# 🏗️ Lab 10: Single-Agent Design Patterns
## Module 10 - Robust Tool-Using Agent with Failure Handling

**Duration:** 30 minutes

**Objectives:**
- Implement layered agent architecture
- Build robust tool execution with fallbacks
- Add observability and logging

**Banking Scenario:** Account lookup agent with failure handling

---

In [2]:
import os
import json
import time
from datetime import datetime
from tenacity import retry, stop_after_attempt, wait_exponential

# =============================================================================
# GOOGLE COLAB SETUP - Add these secrets (click 🔑 icon):
#   - AZURE_OPENAI_KEY: Your API key
#   - AZURE_OPENAI_ENDPOINT: https://xxx.openai.azure.com/
#   - AZURE_OPENAI_DEPLOYMENT: Your model deployment name
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing")
except Exception:
    print("⚠️ Running in DEMO MODE")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Client ready")

✅ Credentials loaded. Model: gpt-4o
✅ Client ready


## Part 1: Layered Agent Architecture

In [3]:
class PlanningLayer:
    """Layer 1: Break tasks into steps"""

    def __init__(self, client):
        self.client = client

    def create_plan(self, task: str) -> list:
        response = self.client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{
                "role": "system",
                "content": "Break the task into numbered steps. Return as JSON array."
            }, {
                "role": "user",
                "content": task
            }],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content).get("steps", [])

class ReasoningLayer:
    """Layer 2: Make decisions based on data"""

    def __init__(self, client):
        self.client = client

    def decide(self, context: dict) -> str:
        response = self.client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{
                "role": "system",
                "content": "Analyze the data and recommend an action."
            }, {
                "role": "user",
                "content": json.dumps(context)
            }]
        )
        return response.choices[0].message.content

print("✅ Planning and Reasoning layers defined")

✅ Planning and Reasoning layers defined


In [4]:
class ToolLayer:
    """Layer 3: Execute tools with retry and fallback"""

    def __init__(self):
        self.tools = {}
        self.fallbacks = {}

    def register(self, name: str, func, fallback=None):
        self.tools[name] = func
        if fallback:
            self.fallbacks[name] = fallback

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(min=1, max=10))
    def _execute_with_retry(self, func, **kwargs):
        return func(**kwargs)

    def execute(self, tool_name: str, **kwargs) -> dict:
        if tool_name not in self.tools:
            return {"error": f"Unknown tool: {tool_name}"}

        try:
            return self._execute_with_retry(self.tools[tool_name], **kwargs)
        except Exception as e:
            if tool_name in self.fallbacks:
                print(f"⚠️ Primary failed, using fallback for {tool_name}")
                return self.fallbacks[tool_name](**kwargs)
            return {"error": str(e), "retryable": False}

print("✅ Tool layer with retry defined")

✅ Tool layer with retry defined


## Part 2: Banking Tools with Fallbacks

In [5]:
# Simulated banking tools
def get_account_primary(account_id: str) -> dict:
    """Primary: Real-time from core banking"""
    # Simulate occasional failures
    import random
    if random.random() < 0.3:
        raise Exception("Core banking timeout")
    return {"account_id": account_id, "balance": 5420.50, "status": "active", "source": "primary"}

def get_account_cache(account_id: str) -> dict:
    """Fallback: Cached data (may be stale)"""
    return {"account_id": account_id, "balance": 5400.00, "status": "active", "source": "cache", "_stale": True}

def get_transactions(account_id: str, days: int = 30) -> dict:
    return {"account_id": account_id, "transactions": [{"amount": -150, "merchant": "Store"}]}

# Register tools
tools = ToolLayer()
tools.register("get_account", get_account_primary, fallback=get_account_cache)
tools.register("get_transactions", get_transactions)

print("✅ Banking tools registered")

✅ Banking tools registered


In [6]:
# Test tool execution with fallback
print("Testing tool execution (may trigger fallback):")
for i in range(5):
    result = tools.execute("get_account", account_id="ACC-123")
    source = result.get("source", "error")
    stale = "(stale)" if result.get("_stale") else ""
    print(f"  Attempt {i+1}: {source} {stale}")

Testing tool execution (may trigger fallback):
  Attempt 1: primary 
  Attempt 2: primary 
  Attempt 3: primary 
  Attempt 4: primary 
  Attempt 5: primary 


## Part 3: Complete Robust Agent

In [7]:
class RobustBankingAgent:
    def __init__(self, client):
        self.planner = PlanningLayer(client)
        self.reasoner = ReasoningLayer(client)
        self.tools = ToolLayer()
        self._setup_tools()
        self.logs = []

    def _setup_tools(self):
        self.tools.register("get_account", get_account_primary, get_account_cache)
        self.tools.register("get_transactions", get_transactions)

    def _log(self, event: str, data: dict):
        self.logs.append({"timestamp": datetime.now().isoformat(), "event": event, **data})

    def execute(self, task: str) -> dict:
        self._log("task_start", {"task": task})

        # Layer 1: Planning
        plan = self.planner.create_plan(task)
        self._log("plan_created", {"steps": len(plan)})

        results = []
        for step in plan:
            # Handle both string and dict formats from LLM
            step_text = step if isinstance(step, str) else str(step.get('step', step.get('description', str(step))))

            # Layer 3: Tool execution
            if "account" in step_text.lower():
                result = self.tools.execute("get_account", account_id="ACC-123")
            elif "transaction" in step_text.lower():
                result = self.tools.execute("get_transactions", account_id="ACC-123")
            else:
                result = {"step": step_text, "status": "skipped"}

            results.append(result)
            self._log("step_complete", {"step": step_text, "success": "error" not in result})

        # Layer 2: Reasoning
        decision = self.reasoner.decide({"task": task, "results": results})
        self._log("task_complete", {"decision": decision[:100]})

        return {"decision": decision, "results": results}

# Test the agent
agent = RobustBankingAgent(client)
result = agent.execute("Check account status and recent transactions for customer inquiry")

print("\n" + "="*50)
print("AGENT DECISION:")
print("="*50)
print(result["decision"])

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

---
## ✅ Lab 10 Complete!

**Key Takeaways:**
- Layered architecture separates concerns
- Retry with exponential backoff handles transient errors
- Fallback tools ensure availability
- Logging enables observability